In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('dataset_phishing.csv')
data.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [2]:
data.info() # further analysis for data cleaning purposes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 89 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         11430 non-null  object 
 1   length_url                  11430 non-null  int64  
 2   length_hostname             11430 non-null  int64  
 3   ip                          11430 non-null  int64  
 4   nb_dots                     11430 non-null  int64  
 5   nb_hyphens                  11430 non-null  int64  
 6   nb_at                       11430 non-null  int64  
 7   nb_qm                       11430 non-null  int64  
 8   nb_and                      11430 non-null  int64  
 9   nb_or                       11430 non-null  int64  
 10  nb_eq                       11430 non-null  int64  
 11  nb_underscore               11430 non-null  int64  
 12  nb_tilde                    11430 non-null  int64  
 13  nb_percent                  114

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
## status of url
le.fit(["phishing", "legitimate"])
transformedPclass = le.transform(data.status[:].values)
data['status'] = transformedPclass

In [4]:
X = data[['https_token', 'ratio_digits_url', 'nb_hyphens', 'nb_dots', 'nb_underscore', 'nb_slash']]
y = data['status']

print("---------------FEATURES---------------------")
print(X)

print("---------------TARGET---------------------")
print(y)

---------------FEATURES---------------------
       https_token  ratio_digits_url  nb_hyphens  nb_dots  nb_underscore  \
0                1          0.000000           0        3              0   
1                1          0.220779           0        1              0   
2                0          0.150794           1        4              2   
3                1          0.000000           0        2              0   
4                1          0.000000           2        2              0   
...            ...               ...         ...      ...            ...   
11425            1          0.000000           0        2              0   
11426            1          0.023810           0        5              0   
11427            0          0.142857           6        2              1   
11428            1          0.000000           0        2              0   
11429            1          0.085954           0       24             18   

       nb_slash  
0             3  
1     

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5)

In [7]:
knn.fit(X_train.values, y_train)

KNeighborsClassifier()

In [8]:
knn.score(X_test.values,y_test)

0.7823652904128762

In [9]:
#url_to_predict = 'https://www.coursera.org/learn/python-machine-learning/lecture/MwsUM/k-nearest-neighbors-classification'
# url_to_predict = 'https://app.simplenote.com/publish/XhRdVc'
#url_to_predict = 'https://stackoverflow.com/questions/24878174/how-to-count-digits-letters-spaces-for-a-string-in-python'
# url_to_predict = 'https://www.lionsgateplus.com/es/es/movies'
# url_to_predict = 'https://www.simplilearn.com/tutorials/python-tutorial/python-if-else-statement'
url_to_predict = 'https://tubitv.com/category/peliculas_en_espanol'
def countDigits(string):
    alpha = 0
    for i in string:
        if (i.isalpha()):
            alpha+=1
    return alpha

def extract_features(url_to_predict):
  # grab how many hyphens
    hyphens = url_to_predict.count('-')
    dots = url_to_predict.count('.')
    slashes = url_to_predict.count('/')
    underscore = url_to_predict.count('_')
    ratio_of_digits = countDigits(url_to_predict)/len(url_to_predict)
    https_token = url_to_predict.__contains__('https')
    if (https_token):
        https_token = 1
    else:
        https_token = 0
    print("There are:")
    print("Hyphens: ", hyphens)
    print("Dots: ", dots)
    print("Slashes: ", slashes)
    print("Underscore: ", underscore)
    print("Ratio of digits: ", ratio_of_digits)
    print("https token: ", https_token)
    
    return https_token, ratio_of_digits, hyphens, dots, underscore, slashes
    
https_token, ratio_digits_url, nb_hyphens, nb_dots, nb_underscore, nb_slash = extract_features(url_to_predict)


url_prediction = knn.predict([[https_token, ratio_digits_url, nb_hyphens, nb_dots, nb_underscore, nb_slash]])
print(url_prediction)

There are:
Hyphens:  0
Dots:  1
Slashes:  4
Underscore:  2
Ratio of digits:  0.8333333333333334
https token:  1
[0]


In [10]:
list(le.inverse_transform(url_prediction))

['legitimate']